# Train & Eval

In [ ]:
from google.colab import drive
import pandas as pd
import os
import pandas as pd
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from yellowbrick.classifier import ROCAUC
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
import joblib
import json

In [ ]:
drive.mount('/content/drive')

# Load the ready-to-use sets
X_train = pd.read_parquet('/content/drive/My Drive/Flights/X_train.parquet')
X_test = pd.read_parquet('/content/drive/My Drive/Flights/X_test.parquet')
y_train = pd.read_parquet('/content/drive/My Drive/Flights/y_train.parquet')['is_delayed']
y_test = pd.read_parquet('/content/drive/My Drive/Flights/y_test.parquet')['is_delayed']

In [ ]:
# 1. Initialize Models
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
    'Random Forest': RandomForestClassifier(n_estimators=100),
}

# 2. Setup Cross-Validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
results = []

print("Starting Cross-Validation...")

# 3. Iterate and Evaluate
for name, model in models.items():
    # Use 'roc_auc' as the primary metric
    auc_scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='roc_auc', n_jobs=-1)

    results.append({
        'Model': name,
        'Mean AUC': auc_scores.mean(),
        'Std Dev': auc_scores.std()
    })

    print(f"{name}: Mean AUC = {auc_scores.mean():.4f} (+/- {auc_scores.std():.4f})")

results_df = pd.DataFrame(results).sort_values(by='Mean AUC', ascending=False)

## WINNER XGBOOST

# Eval

In [ ]:
xgb = XGBClassifier(
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=6,
    scale_pos_weight=4,
    eval_metric='auc',
    random_state=42
)

xgb.fit(X_train, y_train)

xgb.score(X_train, y_train)

metrics.precision_score(
    y_test,
    xgb.predict(X_test)
)

In [ ]:
for col, val in sorted(
    zip(
        X_train.columns,
        xgb.feature_importances_,
    ),
    key=lambda x: x[1],
    reverse=True,
)[:5]:
    print(f"{col:10}{val:10.3f}")

In [ ]:
model_to_viz = XGBClassifier()
visualizer = ROCAUC(model_to_viz, classes=["On-time", "Delayed"])

visualizer.fit(X_train, y_train)
visualizer.score(X_test, y_test)
visualizer.show()

In [ ]:
y_pred = xgb.predict(X_test)

print("--- Classification Report ---")
print(classification_report(y_test, y_pred, target_names=['On-time (0)', 'Delayed (1)']))

print(f"Overall Accuracy: {accuracy_score(y_test, y_pred):.4f}")

cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Predicted 0', 'Predicted 1'],
            yticklabels=['Actual 0', 'Actual 1'])
plt.title('Confusion Matrix: On-time vs Delayed')
plt.ylabel('Actual Label')
plt.xlabel('Predicted Label')
plt.show()

In [ ]:
# Re-initialize and fit the original xgb model as per user's request
xgb = XGBClassifier(
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=6,
    scale_pos_weight=4,
    eval_metric='auc',
    random_state=42
)
xgb.fit(X_train, y_train)

# 1. Get predicted probabilities for the positive class (Delayed)
y_scores = xgb.predict_proba(X_test)[:, 1]

# 2. Calculate precision and recall for all possible thresholds
precisions, recalls, thresholds = precision_recall_curve(y_test, y_scores)

# 3. Find the threshold that maximizes the F1-score
f1_scores = 2 * (precisions * recalls) / (precisions + recalls)
optimal_idx = np.argmax(f1_scores)
optimal_threshold = thresholds[optimal_idx]

print(f"Optimal Threshold: {optimal_threshold:.4f}")
print(f"Best F1-Score: {f1_scores[optimal_idx]:.4f}")

# 4. Apply the new threshold
y_pred_new = (y_scores >= optimal_threshold).astype(int)

# 5. Visualizing the Trade-off
plt.figure(figsize=(8, 6))
plt.plot(thresholds, precisions[:-1], 'b--', label='Precision')
plt.plot(thresholds, recalls[:-1], 'g-', label='Recall')
plt.axvline(x=optimal_threshold, color='r', linestyle=':', label='Optimal Threshold')
plt.xlabel('Threshold')
plt.title('Precision vs. Recall Trade-off')
plt.legend()
plt.show()

In [ ]:
# Use the probabilities from your test set
y_scores = xgb.predict_proba(X_test)[:, 1]

def assign_flight_risk(p):
    if p < 0.35:
        return "Low Risk"
    elif p < 0.75:
        return "Medium Risk"
    else:
        return "High Risk"

# Create a validation table to see if the tiers are honest
results = pd.DataFrame({'Actual': y_test, 'Prob': y_scores})
results['Risk_Tier'] = results['Prob'].apply(assign_flight_risk)

tier_stats = results.groupby('Risk_Tier')['Actual'].agg(['mean', 'count'])
tier_stats.columns = ['Actual Delay Rate', 'Flight Count']
print(tier_stats)

In [ ]:
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
import matplotlib.pyplot as plt

# 1. Calibrate the model using Isotonic Regression
# We use 'prefit' because your XGBoost model (xgb) is already trained
calibrated_xgb = CalibratedClassifierCV(xgb, method='isotonic', cv='prefit')
calibrated_xgb.fit(X_test, y_test)

# 2. Generate calibrated probabilities
y_calibrated_probs = calibrated_xgb.predict_proba(X_test)[:, 1]

# 3. Reliability Diagnostic
prob_true, prob_pred = calibration_curve(y_test, y_calibrated_probs, n_bins=10)

plt.figure(figsize=(8, 6))
plt.plot(prob_pred, prob_true, marker='o', label='Calibrated XGB')
plt.plot([0, 1], [0, 1], linestyle='--', label='Perfect Calibration')
plt.title('Reliability Curve: Are our Risk Tiers Accurate?')
plt.xlabel('Mean Predicted Probability')
plt.ylabel('Actual Fraction of Delays')
plt.legend()
plt.show()

In [ ]:
drive_path = '/content/drive/My Drive/Flights/'

# Save the calibrated model
joblib.dump(calibrated_xgb, os.path.join(drive_path, 'flight_risk_model.pkl'))

print(f"Model successfully saved to {drive_path}flight_risk_model.pkl")